In [70]:
import numpy as np 
import pandas as pd
import networkx as nx
import random

def random_graph(prob_resp,prob_nuevo):
    """
    prob_resp float    define la probabilidad de que al tomar un nodo este haya sido un respondiente de la encuesta
    prob_nuevo float   define la probabilidad de que un respondiente tenga conexiones con nodos nuevos

    Esta funcion genera graficas aleatorias basadas en el metodo con el que se crearon las networks de emprendimiento
    """
    G=nx.Graph()
    queue=["0"]
    max_index=0
    max_respondents=random.randint(20,30)             #Indicador que nos dice cuantos respondientes puede haber
    respondents=0
    while len(queue)!=0:
        if (random.random()<prob_resp and respondents<max_respondents) or queue[0]=="0":      #Crear conexiones para un respondiente
            respondents+=1
            n=random.randint(6,21)
            new_n=0
            old_n=0
            for i in range(n):                                              #Calcular cuantas conexiones son con nodos nuevos y cuantos con ya existentesz
                if random.random()<prob_nuevo:
                    new_n+=1
                else:
                    old_n+=1
            for i in range(new_n):
                new_node=str(i+max_index+1)
                queue.append(new_node)
                G.add_node(new_node)
                G.add_edge(queue[0],new_node)
            for i in range(old_n):
                node=str(random.randint(0,int(queue[0]))-1)
                G.add_edge(queue[0],node)
            max_index+=new_n
            queue.pop(0)
        else:                                                                #Crear conexiones para los no respondientes
            if random.random()<0.25:
                n=random.randint(1,6)
                for i in range(n):
                    node=str(random.randint(0,int(queue[0]))-1)
                    G.add_edge(queue[0],node)
            queue.pop(0)
    return G
    
G1 = random_graph(0.25,0.55)
G2 = random_graph(0.25,0.55)
G3 = random_graph(0.25,0.55)
G4 = random_graph(0.25,0.55)
G5 = random_graph(0.25,0.55)
G6 = random_graph(0.25,0.55)
G7 = random_graph(0.25,0.55)
G8 = random_graph(0.25,0.55)
G9 = random_graph(0.25,0.55)
G10 = random_graph(0.25,0.55)
#nx.write_graphml(G1,'Random_Graphs/Random_Graph_1.graphml')
#nx.write_graphml(G2,'Random_Graphs/Random_Graph_2.graphml')
#nx.write_graphml(G3,'Random_Graphs/Random_Graph_3.graphml')
#nx.write_graphml(G4,'Random_Graphs/Random_Graph_4.graphml')
#nx.write_graphml(G5,'Random_Graphs/Random_Graph_5.graphml')
#nx.write_graphml(G6,'Random_Graphs/Random_Graph_6.graphml')
#nx.write_graphml(G7,'Random_Graphs/Random_Graph_7.graphml')
#nx.write_graphml(G8,'Random_Graphs/Random_Graph_8.graphml')
#nx.write_graphml(G9,'Random_Graphs/Random_Graph_9.graphml')
#nx.write_graphml(G10,'Random_Graphs/Random_Graph_10.graphml')

In [102]:
#-----------------------
# Graph metrics!
#-----------------------

def degree(G):
    return np.mean([G.degree(x) for x in G.nodes()])

def eccentricity(U):
    return np.mean([nx.eccentricity(U,x) for x in U.nodes()])

def clustering(U):
    return np.mean([nx.clustering(U,x) for x in U.nodes()]) 

def diameter(U):
    return nx.diameter(U)

def radius(U):
    return nx.radius(U)

def avg_shortest_path_length(U):
    return nx.average_shortest_path_length(U)

def transitivity(U):
    return nx.transitivity(U)

def global_efficiency(U):
    return nx.global_efficiency(U)

def small_worldness(U):
    return nx.algorithms.smallworld.sigma(U,niter=1,nrand=2)

def rich_club_coeffs(G):
    t_ok = True
    t = 0
    d = dict()
    while t_ok:
        nodes_large_degree=[]
        for x in G.nodes():
            if G.degree(x)>t:
                nodes_large_degree.append(x)
        core = G.subgraph(nodes_large_degree)
        edges_core = len(core.edges())
        nodes_core = len(core.nodes())
        if nodes_core<=1:
            t_ok = False
            break
        d[t] = (2*edges_core)/(nodes_core*(nodes_core-1))
        t += 1 
    return d  

def max_rich_club(G):
    rich_club=rich_club_coeffs(G)
    max_i=0
    
    for i in range(len(rich_club)):
        if rich_club[i]>rich_club[max_i]:
            max_i=i
    return rich_club[max_i]

def core_ratio(G):
    return len(nx.k_core(G,k=2).nodes())/len(G.nodes())
        
def central_point_dominance(G):
    betwennesses = nx.betweenness_centrality(G)
    b_max = max(betwennesses.values())
    N = len(betwennesses.keys())
    count = 0
    for i, b_i in betwennesses.items():
        count += ( b_max - b_i )/(N-1)
    return count


def spectral_radius(G):
    L = nx.normalized_laplacian_matrix(G)
    e = np.linalg.eigvals(L.A)
    e_abs = [abs(x) for x in e]
    return max(e_abs)



        
metric_function_map = {'Degree': degree,
            'Eccentricity': eccentricity,
            'clustering': clustering,
            'Diámetro': diameter,
            'Radio':radius,
            'Camino más corto promedio':avg_shortest_path_length,
            'Transitividad':transitivity,
            'Eficiencia Global':global_efficiency,
            'Small Worldness':small_worldness,
            'Rich Club Coefficient':max_rich_club,
            'Core Ratio':core_ratio,
            'Central Point Dominance':central_point_dominance,
            'Spectral radius':spectral_radius,
            'Modularidad con pesos (1.0)':[]} 


In [71]:
from numpy.random import choice

def are_adjacent(u,v,G):
    if v not in G.nodes():
        return False
    #elif u in G.neighbors(v):
    #    return True
    elif v in G.neighbors(u):
        return True
    else:
        return False

def random_graph_2(n_evals, prob_out, prob_new):
    '''
    Generator of a random graph, given that n
    Given a specific number of respondents of the questionnaire, and that each of them could have provided 25 responses maximum
    
    Input:
     - n_evals  : number of evaluators responding questionnaire
     - prob_out : probability that a mentionned collaboration is outside of the network of evaluators
    
    '''
    
    G=nx.DiGraph()
    setedges = set()
    
    inside = dict()
    outside = dict()
    
    last_index = n_evals-1
    
    for i in range(n_evals):              #add all evaluators to the graph
        G.add_node(i)
        inside[i] = 0
        
    # evaluator |--> no. of collaborations,   in (0,25)~ normal distribution
    sample_num_evaluations = np.random.normal(loc=12.3, scale=4, size=n_evals)
    list_degrees_evaluators = [int(x) for x in sample_num_evaluations]
    
    for i in inside.keys():
        for j in range(list_degrees_evaluators[i]):
            
            if random.random()<prob_out:           #edge added outside evaluators' list
                if outside == dict():                              # if list outside evaluators is new
                    last_index+=1
                    to = last_index
                    G.add_node(to)
                    G.add_edge(i,to)
                    setedges.add((i,to))
                    outside[to] = 1
                    inside[i] += 1
                else:
                    if random.random()<prob_new:               # if edge goes to a new vertex
                        last_index+=1       
                        to=last_index
                        G.add_node(to)
                        G.add_edge(i,to)
                        setedges.add((i,to))
                        outside[to] = 1
                        inside[i] +=1
                        
                    else:
                        vertices, degrees = [], []
                        for vertex, degree in outside.items():
                            if vertex!=i:
                                vertices.append(vertex)
                                degrees.append(degree+1)
                        s = sum(degrees)
                        w = [x/s for x in degrees]
                        to = choice(vertices, size=1, p=w)[0]
                        if are_adjacent(i,to,G):
                            to = choice(vertices, size=1, p=w)[0]
                        G.add_node(to)
                        G.add_edge(i,to)
                        setedges.add((i,to))
                        outside[to]+=1
                        inside[i]+=1
                        
            else:
                vertices, degrees = [], []
                for vertex, degree in inside.items():
                    if vertex != i:
                        vertices.append(vertex)
                        degrees.append(degree+1)
                s = sum(degrees)
                w = [x/s for x in degrees]
                to = choice(vertices, size=1, p=w)[0]
                if are_adjacent(i,to,G):
                    to = choice(vertices, size=1, p=w)[0]
                G.add_node(to)
                G.add_edge(i,to)
                setedges.add((i,to))
                inside[to]+=1
                inside[i]+=1
                
    #print(n_evaluated_list)
    #print('')
    #print(G.nodes())
    #print('')
    #print(G.edges())
    #print('')
    #print([len(list(G.neighbors(x))) for x in G.nodes()])
    return G         


list_cities = ['Aguascalientes', 'Buenos Aires', 'Ciudad de México', 'Guadalajara', 'Hidalgo',
                  'Madrid', 'Montevideo', 'Oaxaca', 'Sao Paulo', 'Santiago de Chile']
num_evaluators = {'Aguascalientes':19, 'Buenos Aires':31, 'Ciudad de México':36, 'Guadalajara':32, 'Hidalgo':19,
                  'Madrid':37, 'Montevideo':48, 'Oaxaca':36, 'Sao Paulo':28, 'Santiago de Chile':25}
for city in list_cities:
    G= random_graph_2(num_evaluators[city],0.2, 0.5)
    nx.write_graphml(G,'Random_Graphs_Second_Type_Corrected/Random_Graph_'+city+'.graphml')


In [72]:
muestra = pd.read_csv('Data_answers_evaluators/Evaluadores ecosistemas.xlsx - Muestra.csv')             


ags_info=pd.read_csv('Data_answers_evaluators/Evaluadores ecosistemas.xlsx - AGS.csv')             
caba_info=pd.read_csv('Data_answers_evaluators/Evaluadores ecosistemas.xlsx - CABA.csv')             
cdmx_info=pd.read_csv('Data_answers_evaluators/Evaluadores ecosistemas.xlsx - CDMX.csv')             
gdl_info=pd.read_csv('Data_answers_evaluators/Evaluadores ecosistemas.xlsx - GDL.csv')             
hgo_info=pd.read_csv('Data_answers_evaluators/Evaluadores ecosistemas.xlsx - HGO.csv')             
mad_info=pd.read_csv('Data_answers_evaluators/Evaluadores ecosistemas.xlsx - MAD.csv')             
mtv_info=pd.read_csv('Data_answers_evaluators/Evaluadores ecosistemas.xlsx - MVD.csv')             
oax_info=pd.read_csv('Data_answers_evaluators/Evaluadores ecosistemas.xlsx - OAX.csv')             
sao_info=pd.read_csv('Data_answers_evaluators/Evaluadores ecosistemas.xlsx - SAO.csv')             
scl_info=pd.read_csv('Data_answers_evaluators/Evaluadores ecosistemas.xlsx - SCL.csv')  

ags_info.rename(columns={'Unnamed: 1':'Colabs'}, inplace=True)
gdl_info.rename(columns={'Unnamed: 1':'Colabs'}, inplace=True)
hgo_info.rename(columns={'Unnamed: 1':'Colabs'}, inplace=True)
oax_info.rename(columns={'Unnamed: 1':'Colabs'}, inplace=True)


list_cities = ['AGS', 'CABA', 'CDMX', 'GDL', 'Pachuca',
                  'Madrid', 'Montevideo', 'Oaxaca', 'Sao Paulo', 'Santiago']
cities_info = {'AGS': ags_info, 
              'CABA' : caba_info,
              'CDMX' : cdmx_info,
              'GDL' : gdl_info,
              'Pachuca' : hgo_info,
              'Madrid' : mad_info,
              'Montevideo' : mtv_info,
              'Oaxaca' : oax_info,
              'Sao Paulo' : sao_info,
              'Santiago' : scl_info}

muestra

,Unnamed: 0,Madrid,Madrid SG,Madrid Typeform,CDMX,Santiago,CABA,Sao Paulo,Montevideo,Oaxaca,GDL,Pachuca,AGS
0,Muestra,38,16,24,51,30,36,34,59,36,32,21,19
1,Nodes,239,120,156,299,195,228,216,198,149,187,125,96
2,Edges,453,166,205,547,385,500,364,767,326,474,254,233
3,Promedio colabs. por participante,12.13,10.13,8.63,12.33,13.04,13.48,10.38,13.4,6.39,10.81,9.52,8.21
4,% Muestra,15.90%,13.33%,15.38%,17.06%,15.38%,15.79%,15.74%,29.80%,24.16%,17.11%,16.80%,19.79%


In [103]:
def check_graph(G):
    '''
    we make sure there are no nodes with   G.in_degree(x) , G.out_degree(x) == 0,0
    '''
    
    to_remove = []
    for x in G.nodes():
        if G.in_degree(x) + G.out_degree(x) == 0:
            to_remove.append(x)
            #print('pinguito!')
    #print(to_remove)
    for x in to_remove:
        G.remove_node(x)
        #print('quitamos un pinguito')
    for x in G.nodes():
        if (G.in_degree(x), G.out_degree(x))==(0,0):
            print('achis achis los mariachis')
    #print('-----')
    
    return G




def eliminate_small_components(G):
    '''
    We make sure there are no disconnected components. 
    '''
    to_remove = []
    
    for x in G.nodes():
        if len(list(nx.node_connected_component(nx.to_undirected(G),x)))< 20 :
            to_remove.append(x)
    for x in to_remove:
        G.remove_node(x)
    
    if nx.number_connected_components(nx.to_undirected(G))>1:
        print('****')    
    return G
            
    

In [104]:
def random_graph_3(infos, city, prob_out, prob_new):
    '''
    With this function, we are simulating the responses we obtained from each of the surveyed ecosystems/cities. 
    The information we have is, for each evaluator, how many 
    
    Generator of a random graph, given the number of respondents of the questionnaire,
    the number of evaluators of collaborations, and the number of collaborations reported by each of them.
    
    Input:
     - infos : dictionary mapping from city to num_collab information
     - city : name of city to be simulated
     - prob_out : probability that a mentionned collaboration is outside of the network of evaluators
     - prob_new : probability that a mentionned collaboration is a newly mentionned org
     
    '''
    
    G=nx.DiGraph()
    setedges = set()
    
    
    #degrees of nodes inside and outside respondent list
    inside = dict()
    outside = dict()
    
    
    #determine number of organizations that answered and number of evaluators: 
    n_responded = int(muestra[city][0])
    n_evals = len(list(infos[city]['Colabs']))
    last_index = n_responded
    
    
    #number of total nodes, and number of edges:                  #still don't know how to use
    num_nodes = int(muestra[city][1])
    num_edges = int(muestra[city][2])
    
    for i in range(last_index):              #add all evaluators to the graph
        G.add_node(i)
        if i in range(n_evals):
            inside[i] = 0
        else:
            outside[i] = 0
        
    
    for i in inside.keys():
        
        deg_i = infos[city]['Colabs'][i]         # the degree of that node is in the info retrieved
        for j in range(deg_i):
            
            if random.random() < prob_out:           #edge added outside evaluators' list
                if outside == dict():                              # if list outside evaluators is new
                    #last_index+=1
                    to = last_index
                    G.add_node(to)
                    G.add_edge(i,to)
                    setedges.add((i,to))
                    outside[to] = 1
                    inside[i] += 1
                else:
                    if random.random()<prob_new:               # if edge goes to a new vertex
                        last_index+=1       
                        to=last_index
                        G.add_node(to)
                        G.add_edge(i,to)
                        setedges.add((i,to))
                        outside[to] = 1
                        inside[i] +=1
                        
                    else:
                        vertices, degrees = [], []
                        for vertex, degree in outside.items():
                            if vertex!=i:
                                vertices.append(vertex)
                                degrees.append(degree+1)
                        s = sum(degrees)
                        w = [x/s for x in degrees]
                        to = choice(vertices, size=1, p=w)[0]
                        if are_adjacent(i,to,G):
                            to = choice(vertices, size=1, p=w)[0]
                        #G.add_node(to)
                        G.add_edge(i,to)
                        setedges.add((i,to))
                        outside[to]+=1
                        inside[i]+=1
                        
            else:
                vertices, degrees = [], []
                for vertex, degree in inside.items():
                    if vertex != i:
                        vertices.append(vertex)
                        degrees.append(degree+1)
                s = sum(degrees)
                w = [x/s for x in degrees]
                to = choice(vertices, size=1, p=w)[0]
                if are_adjacent(i,to,G):
                    to = choice(vertices, size=1, p=w)[0]
                #G.add_node(to)
                G.add_edge(i,to)
                setedges.add((i,to))
                inside[to]+=1
                inside[i]+=1
                
    #now we have the degree of each evaluator covered, we covered some nodes outside of the evaluator set. 
    available_nodes = n_responded - n_evals
    remaining_nodes = num_nodes - len(list(G.nodes()))
    remaining_edges = num_edges - len(list(G.edges()))

    #if remaining_nodes <= 0:
    #    print('chinjoles')
    #    return check_graph(G)
    #if remaining_edges <= 0:
    #    print('chinjoles')
    #    return check_graph(G)
    
    #for i in range( last_index+1, num_nodes ):
    #    G.add_node(i)
    
    for i in range(n_responded):
        for j in range(2):

            if random.random()<prob_out:           #edge added outside evaluators' list
                if random.random()<prob_new:               # if edge goes to a new vertex
                    last_index+=1       
                    to=last_index
                    G.add_node(to)
                    G.add_edge(i,to)
                    setedges.add((i,to))
                    outside[to] = 1
                    if i in outside.keys():
                        outside[i]+=1  
                    elif i in inside.keys():
                        inside[i]+=1 
                        
                else:
                    vertices, degrees = [], []
                    for vertex, degree in outside.items():
                        if vertex!=i:
                            vertices.append(vertex)
                            degrees.append(degree+1)
                    s = sum(degrees)
                    w = [x/s for x in degrees]
                    to = choice(vertices, size=1, p=w)[0]
                    if are_adjacent(i,to,G):
                        to = choice(vertices, size=1, p=w)[0]
                    #G.add_node(to)
                    G.add_edge(i,to)
                    setedges.add((i,to))
                    outside[to]+=1
                    if i in outside.keys():
                        outside[i]+=1  
                    elif i in inside.keys():
                        inside[i]+=1 
                        
            else:
                vertices, degrees = [], []
                for vertex, degree in inside.items():
                    if vertex != i:
                        vertices.append(vertex)
                        degrees.append(degree+1)
                s = sum(degrees)
                w = [x/s for x in degrees]
                to = choice(vertices, size=1, p=w)[0]
                if are_adjacent(i,to,G):
                    to = choice(vertices, size=1, p=w)[0]
                #G.add_node(to)
                G.add_edge(i,to)
                setedges.add((i,to))
                inside[to]+=1
                if i in outside.keys():
                    outside[i]+=1  
                elif i in inside.keys():
                    inside[i]+=1  
    
    F=eliminate_small_components(G)
    return F
                
            
for city in list_cities:
    G = random_graph_3(cities_info,city,0.8, 0.6)
    nx.write_graphml(G,'Random_Graphs_Third_Type_Corrected/Random_Graph_'+city+'.graphml')

In [105]:
#----------------------
# Monte Carlo
#----------------------

def monte_carlo_for_city(infos, city, prob_out, prob_new, num_trials):
    data = {#'avg strength':[],
            #'weight':[],
            'Degree':[],
            #'Weighted Degree':[],
            'Eccentricity':[],
            'clustering':[],
            'Diámetro':[],
            'Radio':[],
            'Camino más corto promedio':[],
            'Transitividad':[],
            'Eficiencia Global':[],
            'Small Worldness':[],
            'Rich Club Coefficient':[],
            'Core Ratio':[],
            'Central Point Dominance':[],
            'Spectral radius':[],
            'Modularidad con pesos (1.0)':[],
            'Colaboratividad 1':[],
            'Colaboratividad 2':[]} 
    
    for i in range(100):
        G = random_graph_3(infos,city,prob_out, prob_new)
        U = nx.to_undirected(G)
        
        #data['avg strength'].append(0)
        #data['weight'].append(0)
        data['Degree'].append( np.mean([G.degree(x) for x in G.nodes()]) )
        #data['Weighted Degree'].append( np.mean([G.degree(x) for x in G.nodes()]) )
        data['Eccentricity'].append( np.mean([nx.eccentricity(U,x) for x in U.nodes()]) )
        data['clustering'].append( np.mean([nx.clustering(U,x) for x in U.nodes()]) )

        data['Diámetro'].append(  nx.diameter(U)  )
        data['Radio'].append(  nx.radius(U)  )

        data['Camino más corto promedio'].append( nx.average_shortest_path_length(U) )
        data['Transitividad'].append( nx.transitivity(U) )
        data['Eficiencia Global'].append( nx.global_efficiency(U) )
        data['Small Worldness'].append( nx.algorithms.smallworld.sigma(U,niter=1,nrand=2) )
        data['Rich Club Coefficient'].append( max_rich_club(U) )
        data['Core Ratio'].append( core_ratio(U) )
        data['Central Point Dominance'].append(0)
        data['Spectral radius'].append( spectral_radius(U) )
        data['Modularidad con pesos (1.0)'].append(0)
        data['Colaboratividad 1'].append(0)
        data['Colaboratividad 2'].append(0)

        
    #'avg strength', 'weight', 'Weighted Degree'
    df = pd.DataFrame(data, columns = ['Degree', 'Eccentricity', 
                                       'clustering', 'Diámetro', 'Radio', 'Camino más corto promedio', 
                                       'Transitividad', 'Eficiencia Global','Small Worldness',
                                       'Rich Club Coefficient', 'Core Ratio', 'Central Point Dominance', 
                                       'Spectral radius', 'Modularidad con pesos (1.0)', 'Colaboratividad 1', 
                                       'Colaboratividad 2'])
    
    return df


monte_carlo_for_city(cities_info, 'AGS', 0.8, 0.6, 100)

,Degree,Eccentricity,clustering,Diámetro,Radio,Camino más corto promedio,Transitividad,Eficiencia Global,Small Worldness,Rich Club Coefficient,Core Ratio,Central Point Dominance,Spectral radius,Modularidad con pesos (1.0),Colaboratividad 1,Colaboratividad 2
0,2.899225,5.077519,0.054795,6,4,3.609254,0.041379,0.313469,0.607436,0.666667,0.379845,0,1.919551,0,0,0
1,3.309091,5.181818,0.088899,7,4,3.341118,0.102695,0.339520,0.856465,0.833333,0.436364,0,1.904633,0,0,0
2,3.956989,4.225806,0.124964,5,3,2.991117,0.131890,0.376305,0.801110,1.000000,0.569892,0,1.813149,0,0,0
3,3.381818,5.045455,0.101622,6,4,3.321935,0.080610,0.340695,0.792017,1.000000,0.490909,0,1.904256,0,0,0
4,3.114754,4.934426,0.084674,6,4,3.416339,0.075305,0.329664,0.685959,1.000000,0.401639,0,1.918313,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,3.566038,4.735849,0.061427,6,3,3.229470,0.101100,0.350003,0.907403,1.000000,0.471698,0,1.871213,0,0,0
96,3.048387,5.169355,0.070796,6,4,3.443221,0.055241,0.327686,0.574697,0.666667,0.419355,0,1.933990,0,0,0
97,3.105691,4.780488,0.056022,6,3,3.305078,0.046025,0.336754,0.426802,0.666667,0.414634,0,1.879073,0,0,0
98,3.412844,4.926606,0.079268,6,4,3.365953,0.103371,0.337204,0.841490,1.000000,0.449541,0,1.866023,0,0,0


In [106]:
def quadratic(L):
    return sqrt( sum([x**2 for x in L])/len(L) )
    
def arithmetic(L):
    return sum(L)/len(L)

def geometric(L):
    prod = 1
    for x in L:
        prod *=x
    return prod**(1/len(L))

def harmonic(L):
    n = len(L)
    sum_reciprocals = sum([1/x for x in L])
    return n/sum_reciprocals


mean_map = {'quadratic': quadratic, 
           'arithmetic': arithmetic, 
           'geometric': geometric, 
           'harmonic': harmonic}

In [114]:
def colaborativity(G, set_comunication, set_preparation, set_resilience, meantype):
    '''
    Computes the colaborativity of a network G based on sets which determine which metrics are to be computed
    
    - set_communication : set containing the names of the metrics that will measure communication pilar
    - set_preparation : contains metrics that will measure preparation for future collabs pilar
    - set_resilience : contins the names of the metrics that will measure resilience pilar
    '''
    comm, prep, resi = [], [], []
    U = nx.to_undirected(G)
    
    for metric in set_communication:
        comm.append( metric_function_map[metric](U) )
    comm = arithmetic(comm)
    
    for metric in set_preparation:
        prep.append( metric_function_map[metric](U) )
    prep = arithmetic(prep)

    
    for metric in set_resilience:
        resi.append( metric_function_map[metric](U) )
    resi = arithmetic(resi)
        
    #now we compute the mean
    
    return mean_map[meantype]([comm, prep, resi])


G = random_graph_3(cities_info,'GDL',0.8, 0.6)
set_communication = {'Eficiencia Global'}
set_preparation = {'Rich Club Coefficient'}
set_resilience = {'Transitividad'}
meantype = 'arithmetic'

print(colaborativity(G, set_comunication, set_preparation, set_resilience, meantype))

0.29578465300852497


In [100]:
def harmonic(L):
    n = len(L)
    sum_reciprocals = sum([1/x for x in L])
    return n/sum_reciprocals

dictio = {'h':harmonic}

dictio['h']([1,2,23,4])
#harmonic([2,2,2])

2.2303030303030305